<div class="alert alert-block alert-info">
<h1>Lukumäärät ja prosentit value_counts()-funktiolla</h1>

Frekvenssitaulukot ja ristiintaulukoinnit voin laskea crosstab()-toiminnolla, mutta taitavalle käyttäjälle value_counts() on kätevä funktio.

Seuraavassa esittelen value_counts()-funktion käytön niksejä.
</div>

In [1]:
import pandas as pd

# Avaan datan
df=pd.read_excel('https://taanila.fi/data1.xlsx')

# Lisään dataan yhden object-tyyppisen muuttujan
df['työteht_obj'] = df['työteht'].replace({1 : 'Erittäin tyytymätön', 2 : 'Tyytymätön', 3 : 'Siltä väliltä',
                                       4 : 'Tyytyväinen', 5 : 'Erittäin tyytyväinen'})

# Nyt datassa on kokonaisluku (int64), liukuluku (float64) ja object-tyyppisiä muuttujia
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   nro          82 non-null     int64  
 1   sukup        82 non-null     int64  
 2   ikä          82 non-null     int64  
 3   perhe        82 non-null     int64  
 4   koulutus     81 non-null     float64
 5   palveluv     80 non-null     float64
 6   palkka       82 non-null     int64  
 7   johto        82 non-null     int64  
 8   työtov       81 non-null     float64
 9   työymp       82 non-null     int64  
 10  palkkat      82 non-null     int64  
 11  työteht      82 non-null     int64  
 12  työterv      47 non-null     float64
 13  lomaosa      20 non-null     float64
 14  kuntosa      9 non-null      float64
 15  hieroja      22 non-null     float64
 16  työteht_obj  82 non-null     object 
dtypes: float64(7), int64(9), object(1)
memory usage: 11.0+ KB


<div class="alert alert-block alert-info">
<h2>Frekvenssien mukainen järjestys</h2>
</div>

In [2]:
# Oletuksena value_counts() järjestää frekvenssit suurimmasta pienimpään
df['työymp'].value_counts()

3    30
4    23
5    11
2     9
1     9
Name: työymp, dtype: int64

In [3]:
# Tulos ei ole dataframe, mutta to_frame() muuntaa sen dataframeksi
df['työymp'].value_counts().to_frame()

,työymp
3,30
4,23
5,11
2,9
1,9


In [4]:
# ascending-parametrilla järjestän frekvenssit pienimmästä suurimpaan
df['työymp'].value_counts(ascending = True).to_frame()

,työymp
1,9
2,9
5,11
4,23
3,30


<div class="alert alert-block alert-info">
<h2>Muuttujan arvojen mukainen järjestys</h2>
</div>

In [5]:
# sort_index() järjestää muuttujan arvot pienimmästä suurimpaan
df['työymp'].value_counts().sort_index().to_frame()

,työymp
1,9
2,9
3,30
4,23
5,11


In [6]:
# Myös päinvastainen järjestys on mahdollinen
df['työymp'].value_counts().sort_index(ascending = False).to_frame()

,työymp
5,11
4,23
3,30
2,9
1,9


In [7]:
# sort_index() laittaa object-tyypin arvot aakkosjärjestykseen
# Tämä ei yleensä ole haluttu järjestys
df['työteht_obj'].value_counts().sort_index().to_frame()

,työteht_obj
Erittäin tyytymätön,5
Erittäin tyytyväinen,8
Siltä väliltä,29
Tyytymätön,15
Tyytyväinen,25


In [8]:
# Haluamani järjestyksen saan listan ja reindex()-toiminnon avulla
# Tärkeää: listassa pitää olla täsmälleen samat arvot kuin muuttujalla alun perin
tyytyväisyydet = ['Erittäin tyytymätön', 'Tyytymätön', 'Siltä väliltä', 'Tyytyväinen', 'Erittäin tyytyväinen']
df['työteht_obj'].value_counts().reindex(tyytyväisyydet).to_frame()

,työteht_obj
Erittäin tyytymätön,5
Tyytymätön,15
Siltä väliltä,29
Tyytyväinen,25
Erittäin tyytyväinen,8


<div class="alert alert-block alert-info">
<h2>Puuttuvien arvojen näyttäminen</h2>
</div>

In [9]:
# Voin halutessani näyttää puuttuvat arvot
df['koulutus'].value_counts(dropna = False).sort_index().to_frame()

,koulutus
1.0,27
2.0,30
3.0,22
4.0,2
NaN,1


<div class="alert alert-block alert-info">
<h2>Prosenttien näyttäminen</h2>
</div>

In [10]:
# Frekvenssit koulutus-muuttujalle
df1 = df['koulutus'].value_counts().sort_index().to_frame()

# Lisään prosenttisarakkeen
# Prosentit saan parametrilla normalize = True
df1['%'] = df['koulutus'].value_counts(normalize = True) * 100

# Lisään muuttujan arvoille tekstimuotoiset arvot
koulutus = ['Peruskoulu', '2. aste', 'Korkeakoulu', 'Ylempi korkeakoulu']
df1.index = koulutus

# Lisään yhteensä-rivin
df1.loc['Yhteensä'] = df1.sum()

df1

,koulutus,%
Peruskoulu,27.0,33.333333
2. aste,30.0,37.037037
Korkeakoulu,22.0,27.160494
Ylempi korkeakoulu,2.0,2.469136
Yhteensä,81.0,100.000000


<div class="alert alert-block alert-info">
<h2>Luokiteltu jakauma</h2>
</div>

In [11]:
# Luokkarajat
bins = [1500, 2000, 2500, 3000, 8000]

df2 = df['palkka'].value_counts(bins = bins).sort_index().to_frame()

df2.loc['Yhteensä'] = df2.sum()

df2

,palkka
"(1499.999, 2000.0]",19
"(2000.0, 2500.0]",28
"(2500.0, 3000.0]",22
"(3000.0, 8000.0]",13
Yhteensä,82


<div class="alert alert-block alert-info">
<h2>Ristiintaulukointi</h2>
</div>

In [12]:
# Ristiintaulukoinnissa hyödynnän groupby()-toimintoa
# unstack siirtää sukupuolet sarakkeisiin
df3= df.groupby('sukup')['koulutus'].value_counts().sort_index().unstack('sukup')

# Riviotsikot
df3.index = koulutus
df3.loc['Yhteensä'] = df3.sum()

# Sarakeotsikot
df3.columns=['Mies', 'Nainen']

df3

,Mies,Nainen
Peruskoulu,22.0,5.0
2. aste,23.0,7.0
Korkeakoulu,15.0,7.0
Ylempi korkeakoulu,2.0,NaN
Yhteensä,62.0,19.0


<div class="alert alert-block alert-info">
<h2>Samalla asteikolla mitattujen frekvenssit yhteen taulukkoon</h2>
</div>

In [13]:
# Ensimmäisen muuttujan frekvenssit
df4 = df['johto'].value_counts(sort = False, normalize = True).to_frame()

# Lisään muiden muuttujien frekvenssit
df4['työtov'] = df['työtov'].value_counts(sort = False, normalize = True)
df4['työymp'] = df['työymp'].value_counts(sort = False, normalize = True)
df4['palkkat'] = df['palkkat'].value_counts(sort = False, normalize = True)
df4['työteht'] = df['työteht'].value_counts(sort = False, normalize = True)

# Asteikon arvot tekstimuodossa aiemmin määritellystä tyytyväisyydet-listasta
df4.index = tyytyväisyydet

df4.loc['Yhteensä'] = df4.sum()

# n-arvot sarakeotsikoihin
for var in df4.columns:
    df4 = df4.rename(columns = {var : var + ', n=' + str(df[var].count())})

df4 * 100

,"johto, n=82","työtov, n=81","työymp, n=82","palkkat, n=82","työteht, n=82"
Erittäin tyytymätön,8.536585,NaN,10.975610,40.243902,6.097561
Tyytymätön,19.512195,3.703704,10.975610,23.170732,18.292683
Siltä väliltä,36.585366,19.753086,36.585366,23.170732,35.365854
Tyytyväinen,28.048780,43.209877,28.048780,12.195122,30.487805
Erittäin tyytyväinen,7.317073,33.333333,13.414634,1.219512,9.756098
Yhteensä,100.000000,100.000000,100.000000,100.000000,100.000000


Katso https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html